# Weather data fetch tests
This notebook will walk through how to get forecasted weather data for a location. 

the weather api used is from weather.gov: https://www.weather.gov/documentation/services-web-api

# the weather fetching process is:
1.  get the lat long for the sites
2.  get the grid points
3.  call the for the appropriate forecast 
4.  convert the json payload into a dataframe 
5.  put it all together


# 1 Get lat long for each site
Using google maps, I typed in the locations and then asked for the lat and long of that spot on the map. Here are the lat long coordinates for our locations
![caltech](caltech_lat_long.png)

In [1]:
import requests
import pandas as pd
import json
from typing import Dict
from typing import Tuple, Dict

caltech_lat =   34.134785646454844
caltech_lon = -118.11691382579643

jpl_lat = 34.20142342818471
jpl_lon = -118.17126565774107

office_lat = 37.33680466796926
office_lon = -121.90743423142634

# 2 Get grid points
The way the weather api system works is by finding the station's grid points that are closest to the lat long coordinates. The code below gets the grid x and grid y from the lat long coordinates

In [2]:
def get_grid_points(latitude:float, longitude:float)->Tuple[str, int, int]:
    """The way the weather api system works is by finding the station's grid points that are closest to the lat long coordinates. The code below gets the grid x and grid y from the lat long coordinates
    inputs:
        latitude: float - the latitude coordinate of the desired weather location
        longitude: float - the longitude coordinate of the desired weather location
    returns:
        (grid_id, grid_x, grid_y) tuple of the office id, and x and y grid coordinates needed for future weather.gov api calls
    > get_grid_points(34.134785646454844, -118.11691382579643)
    >('LOX', 160, 48)"""
    url = f'https://api.weather.gov/points/{latitude},{longitude}'
    print('get grid points url:', url)

    r = requests.get(url, timeout=60)

    print('status code:', r.status_code)
    payload = r.json()

    grid_id = payload['properties']['gridId']
    grid_x = payload['properties']['gridX']
    grid_y = r.json()['properties']['gridY']
    return grid_id, grid_x, grid_y
cal = get_grid_points(caltech_lat, caltech_lon)
off = get_grid_points(office_lat, office_lon)

get grid points url: https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
get grid points url: https://api.weather.gov/points/37.33680466796926,-121.90743423142634
status code: 200


In [3]:
cal

('LOX', 160, 48)

In [4]:
ct_grid, ct_grid_x, ct_grid_y = get_grid_points(caltech_lat, caltech_lon)
jpl_grid, jpl_grid_x, jpl_grid_y = get_grid_points(jpl_lat, jpl_lon)
office_grid, office_grid_x, office_grid_y = get_grid_points(office_lat, office_lon)

get grid points url: https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
get grid points url: https://api.weather.gov/points/34.20142342818471,-118.17126565774107
status code: 200
get grid points url: https://api.weather.gov/points/37.33680466796926,-121.90743423142634
status code: 200


# 3 get forecast
Once we have the office, gridx and gridy for the location, we can get three different kinds of weather forecasts

    forecast - forecast for 12h periods over the next seven days
    forecastHourly - forecast for hourly periods over the next seven days
    forecastGridData - raw forecast data over the next seven days



In [5]:
# deprecated
'''
def get_weather_forecast(office, gridX, gridY, forecast_type='forecast'):
    if forecast_type not in ['12_hour', '7day_hourly', '7day_grid']:
        print(f'{forecast_type} is not a valid option. please select 12_hour, 7day_hourly, or 7day_grid', )
        pass
    
    forecast_urls = {'12_hour':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast',
                     '7day_hourly':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly',
                     '7day_grid':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'}
    
    url = forecast_urls[forecast_type]
    print(url)
    r = requests.get(url)
    return r.json()
'''


"\ndef get_weather_forecast(office, gridX, gridY, forecast_type='forecast'):\n    if forecast_type not in ['12_hour', '7day_hourly', '7day_grid']:\n        print(f'{forecast_type} is not a valid option. please select 12_hour, 7day_hourly, or 7day_grid', )\n        pass\n    \n    forecast_urls = {'12_hour':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast',\n                     '7day_hourly':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly',\n                     '7day_grid':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'}\n    \n    url = forecast_urls[forecast_type]\n    print(url)\n    r = requests.get(url)\n    return r.json()\n"

In [6]:
import time
def get_weather_forecast(office:str, gridX:int, gridY:int)->Dict:
    """forecast for 12h periods over the next seven days
    args:
        office: str - the 3 letter code for the weather station
        gridX: int - the X coordinate for the weather grid
        gridY: int - the Y coordinate for the weather grid
    returns:
        dict - the json payload
    """
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast'
    print(url)
    r = requests.get(url, headers={'Accept': 'application/geo+json'}, timeout=60)
    if r.status_code != 200:
        for i in range(2):
            time.sleep(0.3)
            r = requests.get(url, headers={'Accept': 'application/geo+json'})
            if r.status_code == 200:
                break
        if r.status_code != 200:
            print('Failed to get weather forecast')
            return {}
    return r.json()

In [7]:
def get_hourly_weather_forecast(office:str, gridX:int, gridY:int) -> Dict:
    """forecast for hourly periods over the next seven days
    args:
        office: str - the 3 letter code for the weather station
        gridX: int - the X coordinate for the weather grid
        gridY: int - the Y coordinate for the weather grid
    returns:
        dict - the json payload
    """
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly'
    print(url)
    r = requests.get(url)
    if r.status_code != 200:
        for i in range(2):
            time.sleep(0.3)
            r = requests.get(url, headers={'Accept': 'application/geo+json'})
            if r.status_code == 200:
                break
        if r.status_code != 200:
            print('Failed to get weather forecast')
            return {}
    return r.json()


In [8]:
def get_raw_weather_forecast(office, grid_x, grid_y):
    url = f'https://api.weather.gov/gridpoints/{office}/{grid_x},{grid_y}'
    print(url)
    r = requests.get(url, headers={'Accept': 'application/geo+json'}, timeout=60)
    if r.status_code != 200:
        for i in range(2):
            time.sleep(0.3)
            r = requests.get(url, headers={'Accept': 'application/geo+json'})
            if r.status_code == 200:
                break
        if r.status_code != 200:
            print('Failed to get weather forecast')
            return None
    return r.json()


In [9]:
#forecast_7day_hourly

In [10]:
today = get_weather_forecast(ct_grid, ct_grid_x, ct_grid_y)
today.keys()

https://api.weather.gov/gridpoints/LOX/160,48/forecast


dict_keys(['@context', 'type', 'geometry', 'properties'])

In [11]:
today['properties']['periods'][0]

{'number': 1,
 'name': 'Today',
 'startTime': '2024-04-13T09:00:00-07:00',
 'endTime': '2024-04-13T18:00:00-07:00',
 'isDaytime': True,
 'temperature': 61,
 'temperatureUnit': 'F',
 'temperatureTrend': None,
 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 90},
 'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 13.333333333333334},
 'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 91},
 'windSpeed': '15 mph',
 'windDirection': 'S',
 'icon': 'https://api.weather.gov/icons/land/day/rain,90?size=medium',
 'shortForecast': 'Rain',
 'detailedForecast': 'Rain before 5pm, then rain showers. Cloudy, with a high near 61. South wind around 15 mph, with gusts as high as 25 mph. Chance of precipitation is 90%. New rainfall amounts between a quarter and half of an inch possible.'}

# 4 convert json to df and process

In [12]:
def create_forecast_df(json_forecast):
    forecast = json_forecast['properties']['periods']
    forecast_df = pd.DataFrame(forecast)
    forecast_df['startTime'] = pd.to_datetime(forecast_df['startTime'], utc=True)
    forecast_df['endTime'] = pd.to_datetime(forecast_df['endTime'], utc=True)
    return forecast_df
    
today_weather_df = create_forecast_df(today)

In [13]:
import numpy as np
def convert_weather_values(forecast_df):
    """converts values of temperature, dew point and others from C to F"""

    assert set(['temperature', 'dewpoint', 'windSpeed', 'probabilityOfPrecipitation', 'relativeHumidity']).issubset(forecast_df.columns), f"missing columns for weather values in forecast_df.columns = {forecast_df.columns}"

    # convert temperature to degF and degC
    forecast_df.rename(columns={'temperature': 'temperature_degF'}, inplace=True)
    forecast_df['temperature_degC'] = forecast_df['temperature_degF'].apply(lambda x: np.round((x - 32) * 5.0 / 9.0, 2))

    # convert dewpoint temperature to degF and degC
    forecast_df['dewpoint_degC'] = forecast_df['dewpoint'].apply(lambda x: np.round(x['value'], 2))
    forecast_df['dewpoint_degF'] = forecast_df['dewpoint_degC'].apply(lambda x: np.round(x * 9.0 / 5.0 + 32, 2))

    # extract value from windSpeed column and convert to int
    forecast_df['wind_speed_mph'] = forecast_df['windSpeed'].apply(lambda x: int(x.split()[0]))

    # extract value from json formatted columns
    forecast_df['probabilityOfPrecipitationPercent'] = forecast_df['probabilityOfPrecipitation'].apply(
        lambda x: x['value'])
    forecast_df['relative_humidity_%'] = forecast_df['relativeHumidity'].apply(lambda x: x['value'])
    return forecast_df
# convert_weather_values(today_weather_df)

In [14]:
def convert_today_weather_to_hourly(half_day_df):
    """convert the half day dataframe and expand it to each hour of the forecast

    > half_day = get_weather_forecast(ct_grid, ct_grid_x, ct_grid_y)
    > full_day = convert_today_weather_to_hourly(half_day)
    > full_day.colums
    > ['temperature', 'dewpoint', 'windSpeed', 'relativeHumidity',
       'probabilityOfPrecipitation', 'time_utc', 'time_local']

    """
    start_date = half_day_df['startTime'].min().date()
    end_date = start_date + pd.Timedelta("1d")
    print(start_date, end_date)
    df = pd.DataFrame(index=pd.date_range(start_date, end_date, inclusive='both', freq='h', tz=0),
                      columns=half_day_df.columns.tolist())
    df['time_utc'] = df.index
    df['time_local'] = df.index.tz_convert('US/Pacific')
    i = 0
    # for each hour in the 24 hour day
    for j in range(len(df)):
        # for each hour, apply the 6 hour forecast to that period

        # get the latest forecast
        if half_day_df.loc[i, 'startTime'] < df.index[j]:
            i += 1
        # apply the forecast in row i to the hourly in row j
        df.loc[pd.to_datetime(df.index)[j], :] = half_day_df.loc[i, :]
    print(df.columns)
    df = convert_weather_values(df)

    return df.iloc[:24] # return only the 24 hour period

convert_today_weather_to_hourly(today_weather_df)

2024-04-13 2024-04-14
Index(['number', 'name', 'startTime', 'endTime', 'isDaytime', 'temperature',
       'temperatureUnit', 'temperatureTrend', 'probabilityOfPrecipitation',
       'dewpoint', 'relativeHumidity', 'windSpeed', 'windDirection', 'icon',
       'shortForecast', 'detailedForecast', 'time_utc', 'time_local'],
      dtype='object')


,number,name,startTime,endTime,isDaytime,temperature_degF,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,...,shortForecast,detailedForecast,time_utc,time_local,temperature_degC,dewpoint_degC,dewpoint_degF,wind_speed_mph,probabilityOfPrecipitationPercent,relative_humidity_%
2024-04-13 00:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 01:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 02:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 03:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 04:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 05:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 06:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 07:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 08:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 09:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91


In [15]:
ct_hourly_forecast = get_hourly_weather_forecast(ct_grid, ct_grid_x, ct_grid_y)
ct_hourly_forecast['properties']['periods'][0]

https://api.weather.gov/gridpoints/LOX/160,48/forecast/hourly


{'number': 1,
 'name': '',
 'startTime': '2024-04-13T09:00:00-07:00',
 'endTime': '2024-04-13T10:00:00-07:00',
 'isDaytime': True,
 'temperature': 52,
 'temperatureUnit': 'F',
 'temperatureTrend': None,
 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 49},
 'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 7.222222222222222},
 'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 79},
 'windSpeed': '15 mph',
 'windDirection': 'ESE',
 'icon': 'https://api.weather.gov/icons/land/day/rain,49?size=small',
 'shortForecast': 'Chance Light Rain',
 'detailedForecast': ''}

In [16]:
def create_hourly_forecast_df(json_forecast:dict)->pd.DataFrame:
    # grab each hourly entry in the forecast
    assert 'properties' in json_forecast.keys()

    forecast = json_forecast['properties']['periods']
    forecast_df = pd.DataFrame(forecast)

    # convert startTime to UTC datetime and set as index
    forecast_df['time_local'] = pd.to_datetime(forecast_df['startTime'], utc=True).dt.tz_convert('US/Pacific')  # , utc=True)
    forecast_df['time_utc'] = pd.to_datetime(forecast_df['startTime'], utc=True)
    forecast_df['time'] = forecast_df['time_utc']
    forecast_df = forecast_df.set_index('time')

    forecast_df = convert_weather_values(forecast_df)


    # drop columns that are no longer needed
    return forecast_df#.drop(
        # columns=['startTime', 'endTime', 'windSpeed', 'number', 'name', 'detailedForecast', 'dewpoint',
        #          'probabilityOfPrecipitation', 'relativeHumidity', 'temperatureTrend', 'temperatureUnit'])

In [17]:
ct_hourly_forecast_df = create_hourly_forecast_df(ct_hourly_forecast)
ct_hourly_forecast_df.head()

,number,name,startTime,endTime,isDaytime,temperature_degF,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,...,shortForecast,detailedForecast,time_local,time_utc,temperature_degC,dewpoint_degC,dewpoint_degF,wind_speed_mph,probabilityOfPrecipitationPercent,relative_humidity_%
time,,,,,,,,,,,,,,,,,,,,,
2024-04-13 16:00:00+00:00,1,,2024-04-13T09:00:00-07:00,2024-04-13T10:00:00-07:00,True,52,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 49}","{'unitCode': 'wmoUnit:degC', 'value': 7.222222...",...,Chance Light Rain,,2024-04-13 09:00:00-07:00,2024-04-13 16:00:00+00:00,11.11,7.22,45.00,15,49,79
2024-04-13 17:00:00+00:00,2,,2024-04-13T10:00:00-07:00,2024-04-13T11:00:00-07:00,True,52,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 49}","{'unitCode': 'wmoUnit:degC', 'value': 6.666666...",...,Chance Light Rain,,2024-04-13 10:00:00-07:00,2024-04-13 17:00:00+00:00,11.11,6.67,44.01,15,49,74
2024-04-13 18:00:00+00:00,3,,2024-04-13T11:00:00-07:00,2024-04-13T12:00:00-07:00,True,55,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 86}","{'unitCode': 'wmoUnit:degC', 'value': 7.777777...",...,Rain,,2024-04-13 11:00:00-07:00,2024-04-13 18:00:00+00:00,12.78,7.78,46.00,15,86,73
2024-04-13 19:00:00+00:00,4,,2024-04-13T12:00:00-07:00,2024-04-13T13:00:00-07:00,True,55,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 86}","{'unitCode': 'wmoUnit:degC', 'value': 7.777777...",...,Rain,,2024-04-13 12:00:00-07:00,2024-04-13 19:00:00+00:00,12.78,7.78,46.00,15,86,74
2024-04-13 20:00:00+00:00,5,,2024-04-13T13:00:00-07:00,2024-04-13T14:00:00-07:00,True,58,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 86}","{'unitCode': 'wmoUnit:degC', 'value': 10}",...,Rain,,2024-04-13 13:00:00-07:00,2024-04-13 20:00:00+00:00,14.44,10.00,50.00,15,86,77


In [18]:
jpl_forecast = get_weather_forecast(jpl_grid, jpl_grid_x, jpl_grid_y)
jpl_forecast_df = create_forecast_df(jpl_forecast)
jpl_forecast_df.head()

https://api.weather.gov/gridpoints/LOX/159,51/forecast


,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,59,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 12.22222...","{'unitCode': 'wmoUnit:percent', 'value': 95}",15 to 20 mph,S,"https://api.weather.gov/icons/land/day/rain,90...",Rain,"Rain before 5pm, then rain showers. Cloudy, wi..."
1,2,Tonight,2024-04-14 01:00:00+00:00,2024-04-14 13:00:00+00:00,False,43,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 80}","{'unitCode': 'wmoUnit:degC', 'value': 11.11111...","{'unitCode': 'wmoUnit:percent', 'value': 93}",0 to 15 mph,SSW,https://api.weather.gov/icons/land/night/rain_...,Rain Showers,"Rain showers. Mostly cloudy, with a low around..."
2,3,Sunday,2024-04-14 13:00:00+00:00,2024-04-15 01:00:00+00:00,True,56,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 80}","{'unitCode': 'wmoUnit:degC', 'value': 10}","{'unitCode': 'wmoUnit:percent', 'value': 92}",5 to 15 mph,SSW,https://api.weather.gov/icons/land/day/tsra_sc...,Showers And Thunderstorms,"A chance of rain showers before 11am, then sho..."
3,4,Sunday Night,2024-04-15 01:00:00+00:00,2024-04-15 13:00:00+00:00,False,44,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 40}","{'unitCode': 'wmoUnit:degC', 'value': 10.55555...","{'unitCode': 'wmoUnit:percent', 'value': 93}",5 to 15 mph,WSW,https://api.weather.gov/icons/land/night/rain_...,Chance Rain Showers then Partly Cloudy,A chance of rain showers before 11pm. Partly c...
4,5,Monday,2024-04-15 13:00:00+00:00,2024-04-16 01:00:00+00:00,True,65,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 9.444444...","{'unitCode': 'wmoUnit:percent', 'value': 93}",5 to 15 mph,WNW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 65. West northwest win..."


In [19]:
office_forecast = get_weather_forecast(office_grid, office_grid_x, office_grid_y)
office_forecast_df = create_forecast_df(office_forecast)
office_forecast_df.head()

https://api.weather.gov/gridpoints/MTR/99,82/forecast


,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,59,F,falling,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 7.222222...","{'unitCode': 'wmoUnit:percent', 'value': 88}",10 to 14 mph,SSE,"https://api.weather.gov/icons/land/day/rain,90...",Rain then Showers And Thunderstorms,"Rain before 5pm, then showers and thunderstorm..."
1,2,Tonight,2024-04-14 01:00:00+00:00,2024-04-14 13:00:00+00:00,False,44,F,rising,"{'unitCode': 'wmoUnit:percent', 'value': 80}","{'unitCode': 'wmoUnit:degC', 'value': 7.222222...","{'unitCode': 'wmoUnit:percent', 'value': 92}",7 to 15 mph,SE,https://api.weather.gov/icons/land/night/tsra_...,Showers And Thunderstorms,Showers and thunderstorms before 5am. Mostly c...
2,3,Sunday,2024-04-14 13:00:00+00:00,2024-04-15 01:00:00+00:00,True,63,F,falling,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 8.333333...","{'unitCode': 'wmoUnit:percent', 'value': 95}",5 to 12 mph,E,https://api.weather.gov/icons/land/day/bkn?siz...,Partly Sunny,"Partly sunny. High near 63, with temperatures ..."
3,4,Sunday Night,2024-04-15 01:00:00+00:00,2024-04-15 13:00:00+00:00,False,45,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 8.333333...","{'unitCode': 'wmoUnit:percent', 'value': 93}",5 to 12 mph,NW,https://api.weather.gov/icons/land/night/bkn?s...,Mostly Cloudy,"Mostly cloudy, with a low around 45. Northwest..."
4,5,Monday,2024-04-15 13:00:00+00:00,2024-04-16 01:00:00+00:00,True,68,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 9.444444...","{'unitCode': 'wmoUnit:percent', 'value': 94}",3 to 12 mph,WNW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 68. West northw..."


# 4 put it all together

In [20]:
# def get_processed_hourly_7day_weather(latitude:float, longitude:float, test_mode:bool=False) -> pd.DataFrame:
#     """given the latitude and longitude of a site, return a processed dataframe of the hourly weather. This is a orchestrator function
#     args:
#         latitude: float - the latitude of the site
#         longitude: float - the longitude of the site
#         test_mode: bool - if test mode is true, pull stored data from disk, otherwise call the apis
#     returns
#         weather_df: pd.DataFrame - the processed dataframe of the hourly 7day weather for that site
#     """
#     if test_mode:
#         return pd.DataFrame({})
#     office, grid_x, grid_y = get_grid_points(latitude, longitude)
#     weather_json = get_hourly_weather_forecast(office, grid_x, grid_y)
#     weather_df = create_hourly_forecast_df(weather_json)
#     return weather_df
# test 
# get_processed_hourly_7day_weather(latitude=caltech_lat, longitude=caltech_lon, test_mode=False).to_csv('../data/test_forecast.csv')

In [21]:
from src.weather import get_processed_hourly_7day_weather
today_weather_df, weather_df = get_processed_hourly_7day_weather(latitude=caltech_lat, longitude=caltech_lon, test_mode=False)

2024-04-13 10:15:26.656 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-04-13 10:15:26.658 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-04-13 10:15:26.659 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-04-13 10:15:26.660 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-04-13 10:15:26.662 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-04-13 10:15:26.663 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-04-13 10:15:26.664 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-04-13 10:15:26.731 
  command:

    streamlit run /home/denny/anaconda3/envs/ev/lib/python3.11/site-packages/ipykernel_launcher

get grid points url: https://api.weather.gov/points/34.134785646454844,-118.11691382579643


2024-04-13 10:15:27.506 No runtime found, using MemoryCacheStorageManager


status code: 200
https://api.weather.gov/gridpoints/LOX/160,48/forecast


2024-04-13 10:15:28.310 No runtime found, using MemoryCacheStorageManager
2024-04-13 10:15:28.391 No runtime found, using MemoryCacheStorageManager


2024-04-13 2024-04-14
https://api.weather.gov/gridpoints/LOX/160,48/forecast/hourly


2024-04-13 10:15:28.977 No runtime found, using MemoryCacheStorageManager


In [22]:
weather_features = ['temperature_degC', 'dewpoint_degC','relative_humidity_%', 'wind_speed_mph']
weather_df[weather_features].head()

,temperature_degC,dewpoint_degC,relative_humidity_%,wind_speed_mph
2024-04-13 00:00:00+00:00,16.11,13.33,91,15
2024-04-13 01:00:00+00:00,16.11,13.33,91,15
2024-04-13 02:00:00+00:00,16.11,13.33,91,15
2024-04-13 03:00:00+00:00,16.11,13.33,91,15
2024-04-13 04:00:00+00:00,16.11,13.33,91,15


In [23]:
today_weather_df.head()

,number,name,startTime,endTime,isDaytime,temperature_degF,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,...,shortForecast,detailedForecast,time_utc,time_local,temperature_degC,dewpoint_degC,dewpoint_degF,wind_speed_mph,probabilityOfPrecipitationPercent,relative_humidity_%
2024-04-13 00:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 01:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 02:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 03:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
2024-04-13 04:00:00+00:00,1,Today,2024-04-13 16:00:00+00:00,2024-04-14 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 90}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...",...,Rain,"Rain before 5pm, then rain showers. Cloudy, wi...",NaT,NaT,16.11,13.33,55.99,15,90,91
